## 2a. Evidence - Compliance - Position

Evidence collected in this section checks for position complience in the Gradient Climber Example

In [ ]:
{
    "tags": ["Reinforcement Learning"],
    "quality_attribute": "Car controlled by model does not exceed position limits.",
    "description": "Regardless of input, the model will not produce output values that cause the vehicle to exit the area of operation set in the operational configuration.",
    "inputs": "Initial random start position",
    "output": "Log with 1 for in-bounds, and 0 for out-of-bounds",
}

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

### Set up scnario test case

In [ ]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 2
print(card.quality_scenarios[qa].identifier)
print(card.quality_scenarios[qa].quality)
print(
    card.quality_scenarios[qa].stimulus,
    "from ",
    card.quality_scenarios[qa].source,
    " during ",
    card.quality_scenarios[qa].environment,
    ". ",
    card.quality_scenarios[qa].response,
    card.quality_scenarios[qa].measure,
)

**A Specific test case generated from the scenario:**

**Data and Data Source:**	Vehicle state (position and velocity) from sensors (or approximated by simulation engine in development)

**Measurement and Condition:**	−1.2≤𝑝𝑜𝑠𝑖𝑡𝑖𝑜𝑛≤0.6 at all times

**Context:**	Normal Operation

### Helper Functions


In [ ]:
MEASURE_NAME = "gradient climber position accuracy"
NUM_TRIALS = 100
POS_MIN = -1.2
POS_MAX = 0.6

In [ ]:
import numpy as np
import gymnasium as gym

In [ ]:
env = gym.make("MountainCar-v0", render_mode="rgb_array")
state, info = env.reset()

In [ ]:
# Discretize the state space (position, velocity)
position_bins = np.linspace(-1.2, 0.6, 20)
velocity_bins = np.linspace(-0.07, 0.07, 20)

# Q-table initialization
q_table = np.load(os.path.join(DATA_DIR, "mountain_car.npy"))


# Discretize the continuous state (position and velocity)
def discretize_state(state):
    position, velocity = state
    position_idx = (
        np.digitize(position, position_bins) - 1
    )  # Position bin index
    velocity_idx = (
        np.digitize(velocity, velocity_bins) - 1
    )  # Velocity bin index
    return position_idx, velocity_idx


# Epsilon-greedy action selection
def choose_action(state):
    position_idx, velocity_idx = discretize_state(state)
    return np.argmax(q_table[position_idx, velocity_idx])

## Compliance

In [ ]:
def evaluate_action(state, action):
    "Return 1 if this is the expected action, return 0 if it is the wrong move, and -1 as an error condition"
    position, velocity = state
    if (position >= POS_MIN) and (position <= POS_MAX):
        return np.bool(True)
    else:
        return np.bool(False)

In [ ]:
def test_position_compliance():
    done = False
    states = []
    test_results = []

    for i in range(NUM_TRIALS):
        state, info = env.reset()
        done = False
        actions = []
        num_steps = 0
        num_ok_steps = 0
        # print(f"run {i}: s={state}")

        while not done:
            # Random action selection
            action = choose_action(state)
            states.append(state)
            # Take the action and get the next state, reward, done flag, and info
            next_state, reward, done, truncated, info = env.step(action)

            result = evaluate_action(state, action)
            if result == True:
                test_results.append(1)
                num_ok_steps += 1
            elif result == False:
                test_results.append(0)
            # Update the state for the next iteration
            state = next_state
            num_steps += 1

        print(f"Completed trial {i}: {num_ok_steps/num_steps}")

    return test_results

In [ ]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the TestSuite.
position_compliance_measurement = ExternalMeasurement(
    MEASURE_NAME, Array, test_position_compliance
)
evidence = position_compliance_measurement.evaluate()

# Inspect value
print(evidence)

# Save to artifact store
evidence.save(force=True, parents=True)